# Audio TTS Datasets

> TTS datasets

In [ ]:
#| default_exp audio.datasets.tts

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *

In [ ]:
#| export
import torch
from torch.utils.data import DataLoader, Dataset
from pytorch_lightning import LightningDataModule, LightningModule

from matplotlib import pyplot as plt

from lhotse import CutSet, RecordingSet, SupervisionSet, Fbank, FbankConfig
from lhotse.dataset import BucketingSampler, OnTheFlyFeatures
from lhotse.dataset.collation import TokenCollater
from lhotse.dataset.vis import plot_batch
from lhotse.recipes import download_librispeech, prepare_librispeech

from pathlib import Path
from pprint import pprint
from typing import List, Dict, Optional, Union

## LibriTTS

### Lhotse-based Base Class
https://github.com/Lightning-AI/lightning/issues/10358
https://colab.research.google.com/drive/1HKSYPsWx_HoCdrnLpaPdYj5zwlPsM3NH

In [ ]:
#| export
class LhotseTTSDataset(Dataset):
    def __init__(self,
                tokenizer=TokenCollater, # text tokenizer
                extractor=OnTheFlyFeatures(Fbank(FbankConfig(num_mel_bins=80))) # feature extractor
                ):
        self.extractor = extractor
        self.tokenizer = tokenizer

    def __getitem__(self, cuts: CutSet) -> dict:
        cuts = cuts.sort_by_duration()
        feats, feat_lens = self.extractor(cuts)
        tokens, token_lens = self.tokenizer(cuts)
        return {"feats_pad": feats, "feats_lens": feat_lens, "tokens_pad": tokens, "tokens_lens": token_lens}

In [ ]:
# tok = TokenCollater()
# ds = LhotseTTSDataset(tok)

### TTS Base Class

In [ ]:
#| export 
class TTSDataset(Dataset):
    def __init__(self,
        tokenizer, # text tokenizer
        num_mel_bins:int=80,  # number of mel spectrogram bins
        sampling_rate:int=16000 # sampling rate
        ):
        self.extractor = OnTheFlyFeatures(Fbank(FbankConfig(num_mel_bins=num_mel_bins, sampling_rate=sampling_rate)))
        self.tokenizer = tokenizer

    def __getitem__(self, cuts: CutSet) -> dict:
        cuts = cuts.sort_by_duration()
        feats, feat_lens = self.extractor(cuts)
        tokens, token_lens = self.tokenizer(cuts)
        return {"feats_pad": feats, "feats_lens": feat_lens, "tokens_pad": tokens, "tokens_lens": token_lens}

### LibriTTS DataModule

In [ ]:
#| export
from lhotse.recipes import download_libritts, prepare_libritts
from nimrod.text.tokenizers import Tokenizer
from nimrod.audio.embedding import EncoDec
from torchaudio.datasets import LIBRITTS
from nimrod.audio.utils import plot_waveform

In [ ]:
#(Waveform, Sample_rate, Original_text, Normalized_text, Speaker_ID, Chapter_ID, Utterance_ID)
ds = LIBRITTS("../data/en", 'test-clean')
print(ds[0])


In [ ]:
plot_waveform(ds[0][0], ds[0][1])

In [ ]:
#| export
class LibriTTSDataModule(LightningDataModule):
    def __init__(self,
        target_dir="/data/en/libriTTS", # where data will be saved / retrieved
        dataset_parts=["dev-clean", "test-clean"], # either full libritts or subset
        output_dir="/home/syl20/slg/nimrod/recipes/libritts/data", # where to save manifest
        num_jobs=0 # num_jobs depending on number of cpus available
    ):
        super().__init__()
        self.save_hyperparameters(logger=False)

    def prepare_data(self,) -> None:
        # takes a while to download from openslr mirror (~15 min each for test/dev-clean)
        download_libritts(target_dir=self.hparams.target_dir, dataset_parts=self.hparams.dataset_parts)

    def setup(self, stage = None):
        self.libri = prepare_libritts(corpus_dir=Path(self.hparams.target_dir) / "LibriTTS", dataset_parts=self.hparams.dataset_parts, output_dir=self.hparams.output_dir, num_jobs=self.hparams.num_jobs)
        if stage == 'fit' or stage == None:
            self.cuts_train = CutSet.from_manifests(**self.libri["dev-clean"])
            self.cuts_test = CutSet.from_manifests(**self.libri["test-clean"])
            self.tokenizer = TokenCollater(self.cuts_train)
            self.tokenizer(self.cuts_test.subset(first=2))
            # self.tokenizer.inverse(*self.tokenizer(self.cuts_test.subset(first=2)))
        if stage == "test":
            self.cuts_test = CutSet.from_manifests(**self.libri["test-clean"])
            self.tokenizer = TokenCollater(self.cuts_test)
            self.tokenizer(self.cuts_test.subset(first=2))
            # self.tokenizer.inverse(*self.tokenizer(self.cuts_test.subset(first=2)))

    def train_dataloader(self):
        train_sampler = BucketingSampler(self.cuts_train, max_duration=300, shuffle=True) #, bucket_method="equal_duration")
        return DataLoader(TTSDataset(self.tokenizer, sampling_rate=24000), sampler=train_sampler, batch_size=None, num_workers=self.hparams.num_jobs)

    def test_dataloader(self):
        test_sampler = BucketingSampler(self.cuts_test, max_duration=400, shuffle=False) #, bucket_method="equal_duration")
        return DataLoader(TTSDataset(self.tokenizer, sampling_rate=24000), sampler=test_sampler, batch_size=None, num_workers=self.hparams.num_jobs)

    @property
    def model_kwargs(self):
        return {
            "odim": len(self.tokenizer.idx2token),
        }

### Usage

In [ ]:
# num_jobs=0 turns parallel computing off within jupyter notebook. Else it fails.
dm = LibriTTSDataModule(
    target_dir="../data/en", 
    dataset_parts="test-clean",
    output_dir="../data/en/LibriTTS/test-clean",
    num_jobs=1
)

In [ ]:
# skip download and use local data folder
# dm.prepare_data()

In [ ]:
# libri = prepare_libritts("../data/en/LibriTTS", dataset_parts="test-clean")

In [ ]:
dm.setup(stage='test')

In [ ]:
test_dl = dm.test_dataloader()
batch = next(iter(test_dl))
print(batch.keys())

In [ ]:
print(batch['feats_pad'].shape)
plt.imshow(batch['feats_pad'][3].transpose(0,1))
print(batch['feats_lens'])
print(batch['tokens_pad'][3], batch['tokens_lens'][3])

In [ ]:
original_sentences = dm.tokenizer.inverse(batch['tokens_pad'], batch['tokens_lens'])
print(original_sentences)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()